In [27]:
import os
import subprocess
import collections
import shutil

import climatools.clirad.info as cliradinfo

In [16]:
CLIRADLW_BANDS = cliradinfo.wavenumber_bands(region='lw')
CLIRADLW_BANDS

{1: [(0, 340)],
 2: [(340, 540)],
 3: [(540, 800)],
 4: [(800, 980)],
 5: [(980, 1100)],
 6: [(1100, 1215)],
 7: [(1215, 1380)],
 8: [(1380, 1900)],
 9: [(1900, 3000)],
 10: [(540, 620)]}

In [5]:
DIR_SRC = os.path.join('/chia_cluster/home/jackyu/radiation/crd',
                       'LW/src',
                       'lblnew_-_nref_-_autoabsth_klin_-_gasc_kdesc')

In [17]:
def get_dir_case(params):
    '''
    Returns the absolute path of the directory in which 
    to run the case with input parameters `params`
    '''   
    template = os.path.join(
        '/chia_cluster/home/jackyu/radiation/crd',
        'LW/examples',
        'separate_g_groups',
        'study__lblnew_g1_threshold',
        '{molecule}',
        'band{band:02d}_wn_{vmin:d}_{vmax:d}',
        'ng_{ng:d}',
        'g_ascending_k_descending',
        'refPTs_{refPTs}',
        'ng_refs_{ng_refs}',
        'getabsth_{getabsth}',
        'absth_{absth}',
        'wgt_{wgt}',
        'wgt_flux_{option_wgt_flux}',
        'wgt_k_{option_wgt_k}',
        'klin_{klin}',
        'crd_{commitnumber}',
        'atmpro_{atmpro}')
    
    nref = len(params['ng_refs'])
    vmin, vmax = CLIRADLW_BANDS[params['band']][0]
    ng = sum(params['ng_refs'])
    refPTs = '__'.join(['P_{}_T_{}'.format(*pt) 
                        for pt in params['ref_pts']])
    ng_refs = '__'.join([str(n) for n in params['ng_refs']])
    getabsth = '__'.join(['auto' for _ in range(nref)])
    absth = '__'.join(['dlogN_uniform' for _ in range(nref)])
    wgt = '__'.join(['_'.join([str(w) for w in wgt_ref]) 
                     for wgt_ref in params['wgt']])
    klin = 'none' if params['klin'] == 0 else params['klin']
    
    return template.format(molecule=params['molecule'],
                    band=params['band'], vmin=vmin, vmax=vmax,
                    ng=ng,
                    refPTs=refPTs,
                    ng_refs=ng_refs,
                    getabsth=getabsth,
                    absth=absth,
                    wgt=wgt,
                    option_wgt_flux=params['option_wgt_flux'],
                    option_wgt_k=params['option_wgt_k'],
                    klin=klin,
                    commitnumber=params['commitnumber'],
                    atmpro=params['atmpro'])

In [22]:
def run_cases(cases_params=None):
    
    procs = []
    for params in cases_params:
        dir_case = get_dir_case(params)
        
        print(dir_case)
        
        try:
            os.makedirs(dir_case)
        except FileExistsError:
            print(params, 'This case already exists.')
            print()
            procs.append(None)
            continue
            
        try:
            os.chdir(dir_case)
            assert os.system('cp {}/*.f .'.format(DIR_SRC)) == 0
        except AssertionError:
            print('Problem copying source code to case directory for '
                  'case',
                  dir_case)
            print()
            procs.append(None)
            continue
            
        
        fname_code = 'lblnew.f'
        
        os.chdir(dir_case)
        enter_input_params(fname_code, params=params)
        
        try:
            os.chdir(dir_case)
            os.system('ifort -g -traceback -fpe0 {} -o lblnew.exe'.format(fname_code))
            assert os.path.exists('lblnew.exe') == True
        except AssertionError:
            print('Problem compiling source code for case',
                  params)
            print()
            procs.append(None)
            continue
        
        proc = subprocess.Popen(['./lblnew.exe'], stdout=subprocess.PIPE)
        
        procs.append(proc)
    
    print()
        
    return procs

In [23]:
# for printing some input parameters
import itertools
import re

def innames():
    return ['vstar', 'nband', 'nv', 'dv',
            'flgh2o',
            'tsfc',
            'pressure_reference_1',
            'temperature_reference_1', 
            'pressure_reference_2', 
            'temperature_reference_2',
            'ighigh', 
            'ng', 
            'nref',
            'p_refs',
            't_refs',
            'ng_refs',
            'absth',
            'wgt',
            'option_wgt_flux',
            'option_wgt_k',
            'option_klin',
            'fac_meank',
            'klin']



def pattern_assign(name):
    return ''' 
    parameter .* :: \s* &? \s* ({} \s* = (.*) \n)
    '''.format(name)

def pattern_data(name):
    return '''
    (data [^/{name}]+ {name}[^,] [^/{name}]+ / ([^/]+) /)
    '''.format(name=name)

def pattern_atmpro():
    return '''
    (atmosphere_profiles/(.*)75_r8.pro)
    '''

def pattern_molecule():
    return '''
    (
    data \s+  
    flgh2o \s*,\s* flgco2 \s*,\s* flgo3 \s*,\s* flgn2o \s*,\s*
    flgch4 \s*,\s* flgo2
    \n 
    \s* \* \s* / \s* 
    (
    [01] \s* , \s* [01] \s* , \s*  [01] \s* , \s* [01] \s* , \s* [01] 
    \s* , \s* [01]
    )
    \s* / 
     )
    '''


def enter_input_params(path_lblnew, params=None):
    molecules = ('h2o', 'co2', 'o3', 'n2o', 'ch4', 'o2')
    
    with open(path_lblnew, mode='r', encoding='utf-8') as f:
        code = f.read()
    
    d_in = collections.defaultdict(dict)
    
    molecule_flags = [1 if m==params['molecule'] else 0 
                      for m in molecules]
    input_value = '   ,   '.join([str(flg) for flg in molecule_flags])
    d_in['molecule']['regex'] = pattern_molecule()
    d_in['molecule']['input_value'] = input_value
    
    vmin, vmax = CLIRADLW_BANDS[params['band']][0]
    vstar = vmin
    nband = vmax - vmin # keeping nv * dv = 1
    d_in['vstar']['regex'] = pattern_assign(name='vstar')
    d_in['vstar']['input_value'] = ' ' + str(vstar) + '_r8'
    d_in['nband']['regex'] = pattern_assign(name='nband')
    d_in['nband']['input_value'] = ' ' + str(nband)
    
    p_refs, t_refs = zip(*params['ref_pts'])
    d_in['p_refs']['regex'] = pattern_assign(name='p_refs')
    d_in['t_refs']['regex'] = pattern_assign(name='t_refs')
    d_in['p_refs']['input_value'] = (
        ' (/ ' + 
        ' , '.join([str(p) + '_r8' for p in p_refs]) +
        ' /) ')
    d_in['t_refs']['input_value'] = (
        ' (/ ' + 
        ' , '.join([str(p) + '_r8' for p in t_refs]) +
        ' /) ')
    
    wgt = [w for wgt_ref in params['wgt'] for w in wgt_ref]
    wgt = list(itertools.zip_longest(*(4 * [iter(wgt)])))
    wgt = [[str(v) + '_r8' for v in row if v != None] for row in wgt]
    wgt = [' , '.join(row) for row in wgt]
    input_value = ',\n     &     '.join(wgt)
    d_in['wgt']['regex'] = pattern_data(name='wgt')
    d_in['wgt']['input_value'] = input_value
    
    'nref'
    nref = len(params['ng_refs'])
    d_in['nref']['regex'] = pattern_assign(name='nref')
    d_in['nref']['input_value'] = str(nref) 
    
    'ng'
    ng = sum(params['ng_refs'])
    d_in['ng']['regex'] = pattern_assign(name='ng')
    d_in['ng']['input_value'] = str(ng)
    
    'ng_refs'
    d_in['ng_refs']['regex'] = pattern_assign(name='ng_refs')
    d_in['ng_refs']['input_value'] = (
        ' (/ ' + 
        ' , '.join([str(n) for n in params['ng_refs']]) +
        ' /) ')    
    
    'option_wgt_flux'
    d_in['option_wgt_flux']['regex'] = pattern_assign(name='option_wgt_flux')
    d_in['option_wgt_flux']['input_value'] = str(params['option_wgt_flux'])
    
    'option_wgt_k'
    d_in['option_wgt_k']['regex'] = pattern_assign(name='option_wgt_k')
    d_in['option_wgt_k']['input_value'] = str(params['option_wgt_k'])
    
    'klin'
    d_in['option_klin']['regex'] = pattern_assign(name='option_klin')
    d_in['option_klin']['input_value'] = str(1) if params['klin'] else str(0)
    
    d_in['klin']['regex'] = pattern_assign(name='klin')
    d_in['klin']['input_value'] = str(params['klin']) + '_r8'
    
    'atmpro'
    d_in['atmpro']['regex'] = pattern_atmpro()
    d_in['atmpro']['input_value'] = params['atmpro']
    
    d_in['tsfc']['regex'] = pattern_assign(name='tsfc')
    d_in['tsfc']['input_value'] = str(params['tsfc']) + '_r8'
    
    for name, d in d_in.items():
        regex = re.compile(d['regex'], re.VERBOSE)
        statement, value = regex.findall(code)[0]
        input_statement = statement.replace(value, d['input_value'])
        code = code.replace(statement, input_statement)

    with open(path_lblnew, mode='w', encoding='utf-8') as f:
        f.write(code)    

    
    

In [47]:
params_1 = {
    'molecule': 'h2o',
    'band': 6,
    'ref_pts': [(600, 250)],
    'ng_refs': [4],
    'wgt': [(.3, .45, .6, .95)],
    'option_wgt_flux': 2,
    'option_wgt_k': 1,
    'klin': 5e-25,
    'commitnumber': '31186b1',
    'atmpro': 'saw',
    'tsfc': 257
}


params_2 = {
    'molecule': 'h2o',
    'band': 6,
    'ref_pts': [(600, 250)],
    'ng_refs': [4],
    'wgt': [(.3, .45, .6, .95)],
    'option_wgt_flux': 2,
    'option_wgt_k': 1,
    'klin': 5e-25,
    'commitnumber': '31186b1',
    'atmpro': 'trp',
    'tsfc': 300
}


params_3 = {
    'molecule': 'h2o',
    'band': 9,
    'ref_pts': [(600, 250), ],
    'ng_refs': [9,],
    'wgt': [9 * (1, )],
    'option_wgt_flux': 2,
    'option_wgt_k': 1,
    'klin': 0,
    'commitnumber': '31186b1',
    'atmpro': 'mls',
    'tsfc': 294
}

In [48]:
cases_params = [params_1, params_2, params_3]

In [49]:
procs = run_cases(cases_params)

/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o/band06_wn_1100_1215/ng_4/g_ascending_k_descending/refPTs_P_600_T_250/ng_refs_4/getabsth_auto/absth_dlogN_uniform/wgt_0.3_0.45_0.6_0.95/wgt_flux_2/wgt_k_1/klin_5e-25/crd_31186b1/atmpro_saw
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o/band06_wn_1100_1215/ng_4/g_ascending_k_descending/refPTs_P_600_T_250/ng_refs_4/getabsth_auto/absth_dlogN_uniform/wgt_0.3_0.45_0.6_0.95/wgt_flux_2/wgt_k_1/klin_5e-25/crd_31186b1/atmpro_trp
/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o/band09_wn_1900_3000/ng_9/g_ascending_k_descending/refPTs_P_600_T_250/ng_refs_9/getabsth_auto/absth_dlogN_uniform/wgt_1_1_1_1_1_1_1_1_1/wgt_flux_2/wgt_k_1/klin_none/crd_31186b1/atmpro_mls



In [34]:
[proc.poll() for proc in procs]

[None]

In [50]:
[shutil.rmtree(get_dir_case(params)) for params in cases_params]
None

In [37]:
[get_dir_case(params) for params in cases_params]

['/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o/band06_wn_1100_1215/ng_4/g_ascending_k_descending/refPTs_P_600_T_250/ng_refs_4/getabsth_auto/absth_dlogN_uniform/wgt_0.3_0.45_0.6_0.95/wgt_flux_2/wgt_k_1/klin_5e-25/crd_2415550/atmpro_saw',
 '/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o/band06_wn_1100_1215/ng_4/g_ascending_k_descending/refPTs_P_600_T_250/ng_refs_4/getabsth_auto/absth_dlogN_uniform/wgt_0.3_0.45_0.6_0.95/wgt_flux_2/wgt_k_1/klin_5e-25/crd_2415550/atmpro_trp',
 '/chia_cluster/home/jackyu/radiation/crd/LW/examples/separate_g_groups/study__lblnew_g1_threshold/h2o/band09_wn_1900_3000/ng_9/g_ascending_k_descending/refPTs_P_600_T_250/ng_refs_9/getabsth_auto/absth_dlogN_uniform/wgt_1_1_1_1_1_1_1_1_1/wgt_flux_2/wgt_k_1/klin_none/crd_2415550/atmpro_mls']